In [1]:
import pandas as pd
import datetime as dt

pd.options.display.max_columns=1000
pd.options.display.max_rows = 100

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%run configs.ipynb

In [2]:
# parameter cell
file_name = 'oct_21.csv'

In [3]:
# Parameters
file_name = "march_21.csv"


# RRC Prod

## Ingest

In [4]:
production = pd.read_csv(f'{raw_data_folder}/rrc_prod_blackbeard/2021/{file_name}', header = 5)

In [5]:
production.head()

,Multiple Reports,Lease Type,District,RRC Identifier,Field Name,Lease Name,Gas Well ID,Lease Total,Commingle Permit No.,On Hand Beginning of Month,Production,Volume,Code,On Hand End of Month,Formation Production,Volume.1,Code.1
0,NaN,Gas,05,164066.0,DUBOIS (BARNETT),MILES,1,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN
1,NaN,Gas,7B,211352.0,"GRANBURY, N.E. (ATOKA)",LANGDON,2,NaN,5422.0,4,0,NaN,NaN,4,0,NaN,NaN
2,NaN,Gas,7B,212409.0,"GRANBURY, N.E. (ATOKA)","JOHN J. MILES, JR. GAS UNIT",4,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN
3,NaN,Gas,7B,228695.0,"GRANBURY, N.E. (ATOKA)",LUCY UNIT,2,NaN,5699.0,12,0,NaN,NaN,12,731,731,3.0
4,NaN,Gas,7B,228926.0,"GRANBURY, N.E. (ATOKA)","MARSHALL, SAM A",2,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN


## Transform RRC

### Filter to District

In [6]:
#converting data field to string to handle varying schema interpreations 
#(certain months field will be interpreted as float and string depending sample rows pandas uses

production = production.loc[production['District'].astype(str).isin(['8.0', '08', '8'])]

In [7]:
production

,Multiple Reports,Lease Type,District,RRC Identifier,Field Name,Lease Name,Gas Well ID,Lease Total,Commingle Permit No.,On Hand Beginning of Month,Production,Volume,Code,On Hand End of Month,Formation Production,Volume.1,Code.1
7,NaN,Oil,08,32907.0,ARMER (TUBB),"MCCAMEY, G. B. ""A"" (NCT-B)",NaN,NaN,7024.0,182,24,NaN,NaN,206,0,NaN,NaN
8,NaN,Oil,08,20550.0,ARMER (6350),"MCCAMEY, G. B., -A- /NCT-A/",NaN,NaN,7024.0,58,20,NaN,NaN,78,0,NaN,NaN
9,NaN,Oil,08,45314.0,ARMER (6350),RAYDEN MCCAMEY,NaN,NaN,NaN,322,130,NaN,NaN,452,"2,323","2,323",2.0
10,NaN,Oil,08,48393.0,ARMER (6350),LANDLUBBER,NaN,NaN,8026.0,"2,031","9,853","9,528",0.0,"2,356","29,662","29,662",2.0
11,NaN,Oil,08,49425.0,ARMER (6350),6 POUNDER NE,NaN,NaN,8026.0,483,"1,544","1,581",0.0,446,"6,376","6,376",2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472,NaN,Gas,08,160203.0,WADDELL (GRAYBURG),"WADDELL, W.N.",310,NaN,8988.0,0,0,NaN,NaN,0,0,NaN,NaN
473,NaN,Gas,08,190864.0,WADDELL (GRAYBURG),"WADDELL, W.N.",890,NaN,8988.0,0,0,NaN,NaN,0,0,NaN,NaN
474,NaN,Gas,08,190926.0,WADDELL (GRAYBURG),"WADDELL, W.N.",293,NaN,3501.0,16,35,29,1.0,22,189,189,2.0
475,NaN,Gas,08,205775.0,WADDELL (GRAYBURG),"WADDELL, W. N.",1200,NaN,8092.0,10,22,18,1.0,14,517,517,2.0


In [8]:
production.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 466 entries, 7 to 476
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Multiple Reports            33 non-null     object 
 1   Lease Type                  410 non-null    object 
 2   District                    466 non-null    object 
 3   RRC Identifier              466 non-null    float64
 4   Field Name                  466 non-null    object 
 5   Lease Name                  466 non-null    object 
 6   Gas Well ID                 236 non-null    object 
 7   Lease Total                 14 non-null     object 
 8   Commingle Permit No.        278 non-null    float64
 9   On Hand Beginning of Month  460 non-null    object 
 10  Production                  466 non-null    object 
 11  Volume                      217 non-null    object 
 12  Code                        217 non-null    float64
 13  On Hand End of Month        466 non

### Clean Oil Production Columns and Convert to Int

In [9]:
production['Volume'] = production.loc[:,'Volume'].fillna(0).apply(lambda x: int(str(x).replace(',', '')))
production['Volume.1'] = production.loc[:,'Volume.1'].fillna(0).apply(lambda x: int(str(x).replace(',', '')))

### Rename Volume to rrc_oil_volume

In [10]:
production.rename(columns = {'Volume': 'rrc_oil_volume'}, inplace = True)
production.rename(columns = {'Volume.1': 'rrc_gas_volume'}, inplace = True)

In [11]:
production['rrc_oil_volume'].sum()
production['rrc_gas_volume'].sum()

442912

1696586

### Normalize Field and Reservoir Names with Text Cleaning

In [12]:
production["Normalized_Field_Name"] = production['Field Name'].dropna()\
                                                            .apply(lambda x: str(x).split('(')[0]\
                                                                                    .split(',')[0]\
                                                                                    .strip())

In [13]:
production["Normalized_Reservoir_Name"] = production['Field Name'].dropna()\
                                        .apply(lambda x: str(x).split('(')[-1]\
                                                               .strip().replace(')',''))

### Get Date from Production File

In [14]:
def get_date_from_csv(filepath, col_number, row_number):
    str_date = pd.read_csv(filepath, usecols = [col_number]).iloc[row_number].values[0]
    return str_date.strip()

In [15]:
get_date_from_csv(f'{raw_data_folder}/rrc_prod_blackbeard/2021/{file_name}', 3, 1)

'Mar 2021'

### Add Production Date to Production Data Frame

In [16]:
production.insert(0, 'Date', get_date_from_csv(f'{raw_data_folder}/rrc_prod_blackbeard/2021/{file_name}', 3, 1))

In [17]:
production

,Date,Multiple Reports,Lease Type,District,RRC Identifier,Field Name,Lease Name,Gas Well ID,Lease Total,Commingle Permit No.,On Hand Beginning of Month,Production,rrc_oil_volume,Code,On Hand End of Month,Formation Production,rrc_gas_volume,Code.1,Normalized_Field_Name,Normalized_Reservoir_Name
7,Mar 2021,NaN,Oil,08,32907.0,ARMER (TUBB),"MCCAMEY, G. B. ""A"" (NCT-B)",NaN,NaN,7024.0,182,24,0,NaN,206,0,0,NaN,ARMER,TUBB
8,Mar 2021,NaN,Oil,08,20550.0,ARMER (6350),"MCCAMEY, G. B., -A- /NCT-A/",NaN,NaN,7024.0,58,20,0,NaN,78,0,0,NaN,ARMER,6350
9,Mar 2021,NaN,Oil,08,45314.0,ARMER (6350),RAYDEN MCCAMEY,NaN,NaN,NaN,322,130,0,NaN,452,"2,323",2323,2.0,ARMER,6350
10,Mar 2021,NaN,Oil,08,48393.0,ARMER (6350),LANDLUBBER,NaN,NaN,8026.0,"2,031","9,853",9528,0.0,"2,356","29,662",29662,2.0,ARMER,6350
11,Mar 2021,NaN,Oil,08,49425.0,ARMER (6350),6 POUNDER NE,NaN,NaN,8026.0,483,"1,544",1581,0.0,446,"6,376",6376,2.0,ARMER,6350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472,Mar 2021,NaN,Gas,08,160203.0,WADDELL (GRAYBURG),"WADDELL, W.N.",310,NaN,8988.0,0,0,0,NaN,0,0,0,NaN,WADDELL,GRAYBURG
473,Mar 2021,NaN,Gas,08,190864.0,WADDELL (GRAYBURG),"WADDELL, W.N.",890,NaN,8988.0,0,0,0,NaN,0,0,0,NaN,WADDELL,GRAYBURG
474,Mar 2021,NaN,Gas,08,190926.0,WADDELL (GRAYBURG),"WADDELL, W.N.",293,NaN,3501.0,16,35,29,1.0,22,189,189,2.0,WADDELL,GRAYBURG
475,Mar 2021,NaN,Gas,08,205775.0,WADDELL (GRAYBURG),"WADDELL, W. N.",1200,NaN,8092.0,10,22,18,1.0,14,517,517,2.0,WADDELL,GRAYBURG


### Add columns for net production to Trust

In [18]:
production['Net Oil Volume to Trust (RRC)'] = production['rrc_oil_volume'] * 0.50
production['Net Oil Sales Volume to Trust (RRC)'] = production['rrc_oil_volume'] * 0.50 * 0.75

In [19]:
production['Net Gas Volume to Trust (RRC)'] = production['rrc_gas_volume'] * 0.50
production['Net Gas Sales Volume to Trust (RRC)'] = production['rrc_gas_volume'] * 0.50 * 0.75

In [20]:
production

,Date,Multiple Reports,Lease Type,District,RRC Identifier,Field Name,Lease Name,Gas Well ID,Lease Total,Commingle Permit No.,On Hand Beginning of Month,Production,rrc_oil_volume,Code,On Hand End of Month,Formation Production,rrc_gas_volume,Code.1,Normalized_Field_Name,Normalized_Reservoir_Name,Net Oil Volume to Trust (RRC),Net Oil Sales Volume to Trust (RRC),Net Gas Volume to Trust (RRC),Net Gas Sales Volume to Trust (RRC)
7,Mar 2021,NaN,Oil,08,32907.0,ARMER (TUBB),"MCCAMEY, G. B. ""A"" (NCT-B)",NaN,NaN,7024.0,182,24,0,NaN,206,0,0,NaN,ARMER,TUBB,0.0,0.000,0.0,0.000
8,Mar 2021,NaN,Oil,08,20550.0,ARMER (6350),"MCCAMEY, G. B., -A- /NCT-A/",NaN,NaN,7024.0,58,20,0,NaN,78,0,0,NaN,ARMER,6350,0.0,0.000,0.0,0.000
9,Mar 2021,NaN,Oil,08,45314.0,ARMER (6350),RAYDEN MCCAMEY,NaN,NaN,NaN,322,130,0,NaN,452,"2,323",2323,2.0,ARMER,6350,0.0,0.000,1161.5,871.125
10,Mar 2021,NaN,Oil,08,48393.0,ARMER (6350),LANDLUBBER,NaN,NaN,8026.0,"2,031","9,853",9528,0.0,"2,356","29,662",29662,2.0,ARMER,6350,4764.0,3573.000,14831.0,11123.250
11,Mar 2021,NaN,Oil,08,49425.0,ARMER (6350),6 POUNDER NE,NaN,NaN,8026.0,483,"1,544",1581,0.0,446,"6,376",6376,2.0,ARMER,6350,790.5,592.875,3188.0,2391.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472,Mar 2021,NaN,Gas,08,160203.0,WADDELL (GRAYBURG),"WADDELL, W.N.",310,NaN,8988.0,0,0,0,NaN,0,0,0,NaN,WADDELL,GRAYBURG,0.0,0.000,0.0,0.000
473,Mar 2021,NaN,Gas,08,190864.0,WADDELL (GRAYBURG),"WADDELL, W.N.",890,NaN,8988.0,0,0,0,NaN,0,0,0,NaN,WADDELL,GRAYBURG,0.0,0.000,0.0,0.000
474,Mar 2021,NaN,Gas,08,190926.0,WADDELL (GRAYBURG),"WADDELL, W.N.",293,NaN,3501.0,16,35,29,1.0,22,189,189,2.0,WADDELL,GRAYBURG,14.5,10.875,94.5,70.875
475,Mar 2021,NaN,Gas,08,205775.0,WADDELL (GRAYBURG),"WADDELL, W. N.",1200,NaN,8092.0,10,22,18,1.0,14,517,517,2.0,WADDELL,GRAYBURG,9.0,6.750,258.5,193.875


### Write to File

In [21]:
production.to_csv(f'{processed_data_folder}/rrc_prod/{file_name}', index = False)

## Aggregated EDA

In [22]:
production.groupby(['Normalized_Field_Name']).agg({'rrc_oil_volume': 'sum'})\
                                            .sort_values(by = 'rrc_oil_volume', ascending= False)\
                                            .head(15)

,rrc_oil_volume
Normalized_Field_Name,
SAND HILLS,239433
MONAHANS,91439
ARMER,50668
DUNE,16917
UNIVERSITY WADDELL,11408
MARSTON RANCH,10735
KEYSTONE,7384
WADDELL,6066
RUNNING W,2571


In [23]:
production.groupby(['Normalized_Reservoir_Name', 'Normalized_Field_Name']).agg({'rrc_oil_volume': 'sum'}).sort_values(by = 'rrc_oil_volume', ascending= False).sort_index()

rrc_oil_volume
Normalized_Reservoir_Name Normalized_Field_Name                
6350                      ARMER                           50668
7900                      EDWARDS -04-                        0
CLEAR FORK                LEA                               324
                          MONAHANS                            0
                          SAND HILLS                      67040
CLEAR FORK 4070           SAND HILLS                        337
CLEAR FORK, LOWER         MCKEE                               0
CLEAR FORK, MID.          MCKEE                               0
CLEARFORK                 MARSTON RANCH                   10735
                          MONAHANS                        86094
                          RM                                  0
COLBY                     KEYSTONE                         2103
CONSOLIDATED              KEYSTONE                         4214
CORDONA LAKE              CORDONA LAKE                        0
COWDEN, NORTH             COWDEN                            346
DEV.                      CORDONA LAKE                     1863
DEVONIAN                  CORDONA LAKE                      241
                          UNIVERSITY WADDELL              10467
                          WADDELL                           138
DUNE                      DUNE                            16917
ELLENBURGER               KERMIT                            994
                          SAND HILLS                        882
                          UNIVERSITY WADDELL                  0
GRAYBURG                  GOLDSMITH                           0
                          WADDELL                           346
HOLT                      RUNNING W                          85
                          SAND HILLS                          0
JUDKINS                   SAND HILLS                      14422
KERMIT                    KERMIT                            169
MCKEE                     BLOCK 27                            0
                          KERMIT                              0
                          RUNNING W                           0
MCKNIGHT                  SAND HILLS                      43785
MONAHANS, NORTH           MONAHANS                         5345
ORDOVICIAN                SAND HILLS                          0
PENN UPPER                BLOCK 31                          712
PENN.                     G. P. M.                          162
                          UNIVERSITY WADDELL                 36
QUEEN                     GOLDSMITH                           0
SAN ANDRES                C-BAR                               0
                          DELL                                0
                          KEYSTONE                         1067
                          LEA                                44
                          SHAFTER LAKE                      701
SAN ANDRES CONS.          CRAWAR                              0
SAN ANDRES, CON.          GOLDSMITH                           0
SAN ANGELO                LEA                               725
                          SAND HILLS                        497
SAN ANGELO, UPPER         SAND HILLS                      28092
TUBB                      -Y-                                 0
                          ARMER                               0
                          BLOCK B-21                          0
                          RUNNING W                         532
                          SAND HILLS                      31702
TUBB 4500                 CORDONA LAKE                       10
WADDELL                   RUNNING W                         855
                          WADDELL                          5582
WADDELL SD.               UNIVERSITY WADDELL                905
WICHITA-ALBANY            RUNNING W                        1099
                          SAND HILLS                      29711
WICHITA-ALBANY, GAS       SAND HILLS                        509
WOJCIK-MCELROY   